In [3]:
import pandas as pd
import glob
import json

In [3]:
datasets = glob.glob('Datasets/*.xlsx')

In [4]:
fields = {}
writer = pd.ExcelWriter('output3.xlsx', engine='xlsxwriter')

for file_name in datasets:
    dataset = pd.read_excel(file_name, 0, index_col=None, na_values=['NA'])
    field_names = pd.DataFrame(list(dataset))
    # list(dataset.columns.values)
    dataset_name = file_name.split('_')[2]
    field_names[1] = field_names[0].str.split('/').str[0]
    field_names[2] = field_names[0].str.split('/').str[1]
    form = pd.read_excel('Forms/' + dataset_name + '.xls', sheetname='survey', index_col=None)
    form = form[form.name.notnull()]
    field_names.columns = ['field_name', 'related_question', 'option_name']
    
    all_fields = field_names.merge(form, left_on='field_name', right_on='name', how='left', sort=False)
    
    all_fields['field_type'] = all_fields.type.str.split(' ').str[0]
    all_fields.loc[all_fields.option_name.notnull(), 'field_type']= 'select_many_option'
    all_fields['options_list'] = all_fields.type.str.split(' ').str[1]
    all_fields = all_fields[['field_name', 'related_question','option_name','type','label::English','field_type', 'options_list']]
    options = pd.read_excel('Forms/' + dataset_name + '.xls', sheetname='choices', index_col=None)
    options = options[['name','label::English']].drop_duplicates()
    all_fields = all_fields.merge(options, left_on='option_name', right_on='name', how='left', sort=False)
    
    all_fields.to_excel(writer, sheet_name=dataset_name, header=True, index=False)
    # pd.DataFrame(field_names).to_excel(writer, sheet_name=dataset_name, header=False, index=False)
    fields[dataset_name] = field_names

writer.save()

In [6]:
def extract_options(dataset_name):
    admin = pd.read_excel('output3.xlsx', dataset_name, index_col=None, na_values=['NA'])
    select_ones = admin.loc[admin.field_type.isin(['select_one']),['field_name','options_list']]
    so_json = select_ones.to_json(orient='values')
    # dict.fromkeys(seq[, value])
    # obj = {}
    # for question in json.loads(so_json):
    #     obj[question[0]] = question[1]
    so =json.loads(so_json)
    
    keys = map(lambda x: x[0], so)
    values = map(lambda x: x[1], so)
    option_lists = [dict((q[0],q[1]) for q in so)]
    
    with open(dataset_name+'_questions.json', 'w') as outfile:
        json.dump(option_lists, outfile)
        options_df = pd.read_excel('Forms/'+dataset_name+'.xls', 'choices')
        options = options_df[['list_name','name','label::English']].drop_duplicates()
        options.columns= ['list_name','name','label']
        options_json = json.loads(options.to_json(orient='records'))
        qs = pd.Series(map(lambda x: x['list_name'], options_json)).unique()
        obj = [{}]
        for q in qs:
            obj[0][q] = filter(lambda x: x['list_name']==q, options_json)
        with open(dataset_name+'_options.json', 'w') as outfile:
            json.dump(obj, outfile)

In [1]:
def process_dataset(dataset_name): 
    
    questions = pd.read_excel('output3.xlsx', dataset_name, index_col=None, na_values=['NA'])
    questions.loc[questions['label::English_x'].isnull() & questions.option_name.isnull(),'label::English_x'] = questions.loc[questions['label::English_x'].isnull() & questions.option_name.isnull(),'field_name']
    q_json = questions.loc[questions.option_name.isnull(),['field_name', 'label::English_x']].to_json(orient='values')
    header = [dict((q[0],q[1]) for q in json.loads(q_json))]
    names = map(lambda x: x[0], json.loads(q_json))
    labels = map(lambda x: x[1], json.loads(q_json))
    
    datasets = glob.glob('Datasets/*'+dataset_name+'*.xlsx')
    
    data = pd.read_excel(datasets[0], sheetname=0, index_col=None, na_values=['NA'])
    data.columns = list(questions.field_name)
    with open(dataset_name+'_header.json', 'w') as outfile:
        json.dump(header, outfile)
    
    data[names].to_csv(path_or_buf=dataset_name+'.txt', sep='\t', encoding='utf-8', index=False)

In [35]:
process_dataset('Health')

In [36]:
extract_options('Health')

In [4]:
fields = pd.read_excel('output3.xlsx', 'Community', index_col=None, na_values=['NA'])

In [6]:
dataset = pd.read_excel('REACH_ABA_Community_Final_Dataset.xlsx', 'Long', index_col=None, na_values=['NA'])

In [60]:
wide = pd.read_excel('REACH_ABA_Community_Final_Dataset.xlsx', 'Wide', index_col=None, na_values=['NA'])

In [61]:
wide = wide[['id','Relevant Settlement','KOATUU','Relevant Community','Extracted Community']]

In [62]:
d1 = dataset.merge(fields, left_on='Column', right_on='field_name', how='left')
d2 = d1.merge(wide, left_on='Row', right_on='id', how='left')

In [63]:
writer = pd.ExcelWriter('REACH_ABA_Community_Processed.xlsx', engine='xlsxwriter')

In [64]:
d2.to_excel(writer, sheet_name='Long', header=True, index=False)

In [76]:
writer.save()

In [71]:
question_labels = fields[fields['label::English_x'].notnull()][['field_name', 'label::English_x']]

In [73]:
d3 = d2.merge(question_labels, left_on='related_question', right_on='field_name', how='left')

In [74]:
writer = pd.ExcelWriter('REACH_ABA_Community_Processed_2.xlsx', engine='xlsxwriter')

In [75]:
d3.to_excel(writer, sheet_name='Long', header=True, index=False)